# 1B2. Práctica: Cálculo de frecuencias a nivel de palabras

### Alumno: **Luis Fernando Izquierdo Berdugo**
### Materia: **Procesamiento de Información**
### Fecha: **20 de Agosto de 2024**

**Instrucciones**:

Crear un notebook para preprocesar el texto y convertirlo a minúsculas, quitar acentos y los siguientes caracteres:

`;:,.\\-\"'/()[]¿?¡!{}~<>|«»-—’\t\n\r`

1. Calcular las frecuencias a nivel de palabras de los siguientes documentos.
- Libro 1
- Libro 2
Generar los histogramas para cada archivo.

2. En el notebook, responder las siguientes preguntas:

- ¿Cuántas palabras hay en cada texto?
- ¿Cuántas palabras diferentes (palabras únicas) hay en el texto?
- ¿Cuántas son las palabras con frecuencia 1 (hapax legomena) en el texto? 
- Generar los histogramas de todos los documentos.  Para generar los histogramas, ordenar las palabras de acuerdo a la frecuencia de mayor a menor. Si el número de datos es excesivo y difícil de visualizar, presente los datos que considere convenientes, para que se aprecien, de forma adecuada, los datos graficados.

In [20]:
import re
import unicodedata

def remove_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')

file = open("libro_1.txt", "r")
libro_1 = file.read()
file.close()
libro_1 = remove_accents(libro_1)
libro_1 = libro_1.lower()
libro_1 = re.sub(r"[;:,.\-\"'/\(\)\[\]¿?¡!\{\}~<>|«»-—’\t\n\r]", "", libro_1)

file = open("libro_2.txt", "r")
libro_2 = file.read()
file.close()
libro_2 = remove_accents(libro_2)
libro_2 = libro_2.lower()
libro_2 = re.sub(r"[;:,.\-\"'/\(\)\[\]¿?¡!\{\}~<>|«»-—’\t\n\r]", "", libro_2)

file = open("text_1.txt", "r")
text_1 = file.read()
file.close()
text_1 = text_1.lower()
text_1 = re.sub(r"[;:,.\-\"'/\(\)\[\]¿?¡!\{\}~<>|«»-—’\t\n\r]", "", text_1)

file = open("text_2.txt", "r")
text_2 = file.read()
file.close()
text_2 = text_2.lower()
text_2 = re.sub(r"[;:,.\-\"'/\(\)\[\]¿?¡!\{\}~<>|«»-—’\t\n\r]", "", text_2)

file = open("text_3.txt", "r")
text_3 = file.read()
file.close()
text_3 = text_3.lower()
text_3 = re.sub(r"[;:,.\-\"'/\(\)\[\]¿?¡!\{\}~<>|«»-—’\t\n\r]", "", text_3)

file = open("text_4.txt", "r")
text_4 = file.read()
file.close()
text_4 = text_4.lower()
text_4 = re.sub(r"[;:,.\-\"'/\(\)\[\]¿?¡!\{\}~<>|«»-—’\t\n\r]", "", text_4)

file = open("text_5.txt", "r")
text_5 = file.read()
file.close()
text_5 = text_5.lower()
text_5 = re.sub(r"[;:,.\-\"'/\(\)\[\]¿?¡!\{\}~<>|«»-—’\t\n\r]", "", text_5)